In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Flatten, Dense, BatchNormalization, Conv2D, MaxPooling2D, Dropout
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import os
from PIL import Image

2023-04-02 14:04:11.841181: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 14:04:12.623984: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class BirdDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, use_rows, batch_size, csv_file="birds.csv"):
        self.csv_df = pd.read_csv(csv_file)
        self.use_rows = use_rows
        self.batch_size = batch_size 
        self.dataset = pd.DataFrame(self.csv_df).to_numpy()
    
        self.labels = self.dataset[:,2]
        self.img_filepaths = self.dataset[:,1]
        self.class_ids = self.dataset[:,0]
        
        (x,) = self.img_filepaths.shape
        print(self.dataset.shape)
        print(self.img_filepaths[0])
        image = Image.open(self.img_filepaths[0])
        image_array = np.expand_dims(np.asarray(image), axis=0)
        
#         print(image_array)
        print(image_array.shape)
        
#         self.images = []
        
#         for i in range(0, 10):
#             image = Image.open(self.img_filepaths[0])
#             image_array = np.expand_dims(np.asarray(image), axis=0)
#             self.images.append(image_array)
            
#         self.images = np.array(self.images)
        
#         print(self.images.shape)
    
    def __len__(self):
        # batches_per_epoch is the total number of batches used for one epoch
        batches_per_epoch = int(len(self.use_rows) / self.batch_size)
        return batches_per_epoch
    
    def __getitem__(self, index):
        # index is the index of the batch to be retrieved
        batch_ids = self.use_rows[index * self.batch_size: (index + 1) * self.batch_size]
        
        x = None
        y = None
        
#         print("batch index: "+str(index))
#         print("batch ids: " + str(batch_ids))
        
        for curr_id in batch_ids:
            image = Image.open(self.img_filepaths[curr_id]).resize((224,224))
#             image = image.convert("L")
#             np_row = np.asarray(image)
            image_array = np.expand_dims(np.asarray(image),axis=0)
            
            if x is None:
                x = image_array
                y = self.class_ids[curr_id]
            else:
                x = np.vstack((x, image_array))
                y = np.vstack((y, self.class_ids[curr_id]))
                
    
#             print(self.class_ids[curr_id-1])
            
        
#         images = np.array(images)
#         id_labels = np.array(id_labels)
#         print(id_labels)
#         print(images.shape)
#         print(id_labels.shape)
#         print(self.class_ids)
        
#         print(self.class_ids.shape)
        
        
        y = tf.keras.utils.to_categorical(y, num_classes=511)
#         print(x.shape)
#         print(y.shape)
        return x, y            
            
#     def on_epoch_end(self):
#         np.random.shuffle(self.split_ids)
            
            

In [3]:
use_rows = list(range(0,87050))
bird_data_generator = BirdDataGenerator(csv_file="birds.csv", use_rows=use_rows, batch_size=128)
bird_data_generator.__getitem__(0)

(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)


(array([[[[180, 172, 149],
          [186, 178, 155],
          [186, 180, 158],
          ...,
          [181, 180, 198],
          [179, 180, 200],
          [178, 179, 199]],
 
         [[182, 174, 153],
          [190, 182, 161],
          [190, 184, 162],
          ...,
          [182, 181, 199],
          [176, 177, 197],
          [178, 179, 199]],
 
         [[186, 178, 159],
          [195, 187, 168],
          [196, 189, 170],
          ...,
          [184, 183, 199],
          [180, 179, 197],
          [183, 182, 200]],
 
         ...,
 
         [[151, 138, 121],
          [172, 159, 142],
          [178, 165, 146],
          ...,
          [189, 190, 211],
          [191, 189, 210],
          [169, 167, 188]],
 
         [[156, 140, 124],
          [155, 139, 123],
          [161, 148, 131],
          ...,
          [186, 188, 211],
          [182, 185, 204],
          [168, 171, 188]],
 
         [[157, 141, 125],
          [156, 140, 124],
          [167, 154, 137],
   

In [4]:
def bird_cnn_model(csv_file):
    model = Sequential([
        InputLayer(input_shape=(224,224,3)),
        Conv2D(filters=32, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=2),
        Flatten(),
        Dense(511, activation="softmax")
    ])
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics=["accuracy"])
    
    print(model.summary())
    train_rows = list(range(0,81950))
    train_generator = BirdDataGenerator(csv_file=csv_file, use_rows=train_rows, batch_size=64)
    h = model.fit(x=train_generator, epochs=1, verbose=1)
    print(h)
    
#     validation_rows = list(range(84500, 87050))
#     validation_generator = BirdDataGenerator(csv_file=csv_file, use_rows=validation_rows, batch_size=128)
    
    test_rows = list(range(81950, 84500))
    test_generator = BirdDataGenerator(csv_file=csv_file, use_rows=test_rows, batch_size=128)
    model.evaluate(x=test_generator)
    
    return model
    

In [5]:
cnn_model = bird_cnn_model("birds.csv")

2023-04-02 14:04:16.172361: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:04:16.299154: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:04:16.300270: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:04:16.307529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:04:16.309088: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 511)               201473503 
                                                                 
Total params: 201,474,527
Trainable params: 201,474,463


r/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:04:17.656405: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1722] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2023-04-02 14:04:17.657050: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 14:04:17.657160: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3881 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 Ti with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5


None
(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)


2023-04-02 14:04:18.464339: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-02 14:04:20.193380: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-02 14:04:21.037890: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-02 14:04:21.438203: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f38fb76d810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-02 14:04:21.438253: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti with Max-Q Design, Compute Capability 7.5
2023-04-02 14:04:21.601459: I tensorflow/tsl/p

...

1280/1280 [==============================] - 557s 433ms/step - loss: 7.3006 - accuracy: 0.0214
(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)


2023-04-02 14:13:36.819429: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-02 14:13:38.111246: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 665.74MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-02 14:13:38.201220: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.90GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 1/19 [>.............................] - ETA: 29s - loss: 5.7224 - accuracy: 0.0859

2023-04-02 14:13:38.401443: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 665.74MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-02 14:13:38.401545: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


19/19 [==============================] - 19s 955ms/step - loss: 5.3298 - accuracy: 0.1040


In [6]:
cnn_model.save("batch_normalized_1_epoch")

2023-04-02 14:25:44.985467: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 805891968 exceeds 10% of free system memory.
2023-04-02 14:25:45.406861: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 805891968 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: batch_normalized_1_epoch/assets


INFO:tensorflow:Assets written to: batch_normalized_1_epoch/assets
